# Title

### Group 18

Members: Antonie Hille Bram Dorland, Michael Liem, Nynke Sterk, Gergely Turi​

GitHub repository: https://github.com/mtliem98/TIL6022_projectgroup_18

# Contribution Statement

Antonie Hille:

Bram Dorland:

Michael Liem:

Nynke Sterk:

Gergely Turi​:

# Introduction

Modelling the effect of building the Lelylijn on line usage in the surrounding area.

# Research Questions

How many people travel between each station currently?

See how the newly built line would affect the travel time map. How many people have better access to big transport hubs?

How many passengers will use the Lelylijn without any additional demand?​

How does the passenger flow change with the Lelylijn on the surrounding train lines without any additional demand?​

# Data Used

NS dashboard reizigersgedrag, 2024 (https://dashboards.nsjaarverslag.nl/reizigersgedrag)​

GTFS routes, locations (https://gtfs.ovapi.nl/nl/gtfs-nl.zip)

Current and proposed train timetable​ (https://delelylijn.nl/wp-content/uploads/2024/05/28-Meibrief-Lelylijn_280524.pdf)

# Data Pipeline

Import data on all stations and lines.

Import GTFS dataset.

Create L and P-graph.

Use a shortest path algorithm to determine the cost of travelling between two places.

Model travel behaviour and create an OD-matrix.

Verify the above model with the directional data from the NS.

Calculate the projected change in travel times (and create maps and charts on it).

Apply the same travel behaviour model to the new map with the same number of passengers.

# Scale

### Geographical

East of Amsterdam​

Northeast of Utrecht​

North of Deventer​

### Temporal

We take the average daily travellers for the travel demand. Last year’s data is used all the way through the exercise; we do not estimate any changes on the existing infrastructure unrelated to the construction of the new line.

​

# Graphs, questions here

# Conclusion